In [40]:
import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

2.10.0


In [41]:
dataset = pd.read_csv('../datas/RGB_color_labels.csv')
dataset = pd.get_dummies(dataset, columns=['label'])

In [42]:
train_dataset = dataset.sample(frac=0.8, random_state=8) #train_dataset = 80% of total dataset  
#random_state = any int value means every time when you run your program you will get the same output for train and test dataset, random_state is None by default which means every time when you run your program you will get different output because of splitting between train and test varies within 
test_dataset = dataset.drop(train_dataset.index) #remove train_dataset from dataframe to get test_dataset

In [43]:
train_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    train_labels[x] = train_dataset.pop(x)

In [44]:
test_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    test_labels[x] = test_dataset.pop(x)

In [45]:
train_dataset = np.array(train_dataset)
train_labels = np.array(train_labels)
train_labels = np.argmax(train_labels, axis=1)

test_dataset = np.array(test_dataset)
test_labels = np.array(test_labels)
test_labels = np.argmax(test_labels, axis=1)


In [46]:
def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(3, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=(3,)))
    for i in range(1,5):
        model.add(tf.keras.layers.Dense(15, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
    model.add(tf.keras.layers.Dense(11))

    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.build()

    return model

In [47]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 3)                 12        
                                                                 
 dense_7 (Dense)             (None, 15)                60        
                                                                 
 dense_8 (Dense)             (None, 15)                240       
                                                                 
 dense_9 (Dense)             (None, 15)                240       
                                                                 
 dense_10 (Dense)            (None, 15)                240       
                                                                 
 dense_11 (Dense)            (None, 11)                176       
                                                                 
Total params: 968
Trainable params: 968
Non-trainable 

In [48]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 verbose=1,
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 save_freq=10)

# Train the model with the new callback
model.fit(train_dataset, 
          train_labels,  
          epochs=900,
          batch_size = 1000,
          validation_data=(test_dataset, test_labels),
          callbacks=[cp_callback], # Pass callback to training
          shuffle=True)  

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/900
5/5 [==============================] - 2s 88ms/step - loss: 10.2413 - accuracy: 0.1168 - val_loss: 6.7531 - val_accuracy: 0.0802
Epoch 2/900
5/5 [==============================] - 0s 8ms/step - loss: 5.9269 - accuracy: 0.0475 - val_loss: 4.7472 - val_accuracy: 0.0406
Epoch 3/900
5/5 [==============================] - 0s 8ms/step - loss: 4.0346 - accuracy: 0.0891 - val_loss: 3.4346 - val_accuracy: 0.3782
Epoch 4/900
5/5 [==============================] - 0s 8ms/step - loss: 3.1665 - accuracy: 0.3716 - val_loss: 2.8249 - val_accuracy: 0.3782
Epoch 5/900
5/5 [==============================] - 0s 8ms/step - loss: 2.6120 - accuracy: 0.3840 - val_loss: 2.4200 - val_accuracy: 0.4891
Epoch 6/900
5/5 [==============================] - 0s 8ms/step - loss: 2.3510 - accuracy: 0.4052 - val_loss: 2.3736 - val_accuracy: 0.2604
Epoch 7/900
5/5 [==============================] - 0s 7ms/step - loss: 2.3091 - accuracy: 0.2853 - val_loss: 2.2144 - val_accuracy: 0.4495
Epoch 8/900
5/5 [========

In [49]:
model.evaluate(test_dataset, test_labels, verbose=2)

32/32 - 0s - loss: 0.7729 - accuracy: 0.7030 - 76ms/epoch - 2ms/step


[0.7728811502456665, 0.7029703259468079]

In [50]:
# this saves the whole model including the loss function used, everythin etc.
save_path = "../trained_models/model(YOU CLICKED RUN ALL!)"
model.save(f"{save_path}/tensor_model")

INFO:tensorflow:Assets written to: ../trained_models/model1/tensor_model\assets


In [51]:
# this loads the whole model saved
# verbose just means what do you want to see in the outputs(what metrics do you want to see etc.)
new_model = tf.keras.models.load_model(f"{save_path}/tensor_model")
new_model.evaluate(test_dataset, test_labels, verbose=2)

32/32 - 0s - loss: 0.7729 - accuracy: 0.7030 - 416ms/epoch - 13ms/step


[0.7728811502456665, 0.7029703259468079]

In [52]:
# this part saves the weights in a excel file
import xlsxwriter
relative_row_idx = 0 # this is for writing bias
row_idx = 0
max_row_idx = 0
column_idx = 0

workbook = xlsxwriter.Workbook(f"{save_path}/saved_weights_biases.xlsx")
worksheet = workbook.add_worksheet()

In [53]:
for layer in new_model.layers:
    for row_weights in layer.get_weights()[0].T: # the reason I transposed the matrix because tensorflow makes weights in transposed position of matrix
        for weights in row_weights:
            worksheet.write(row_idx, column_idx, weights)
            column_idx += 1

        column_idx += 1
        worksheet.write(row_idx, column_idx, layer.get_weights()[1].T[relative_row_idx]) # the reason I transposed the matrix because tensorflow makes weights in transposed position of matrix
        relative_row_idx += 1
        row_idx += 1
        column_idx = 0

    relative_row_idx = 0
    row_idx += 2
workbook.close()